In [3]:
import pandas as pd
import numpy as np
import vnstock as vn

In [4]:
# Load data
START = "2019-01-01"
END = "2024-02-01"
def load_data(ticker):
        data = vn.stock_historical_data(ticker, START, END)    
        return data
data = load_data('VCB') 
data = data.set_index('time')
data

,open,high,low,close,volume,ticker
time,,,,,,
2019-01-02,34750,35010,34490,34560,1081640,VCB
2019-01-03,34820,34880,33980,34560,1071350,VCB
2019-01-04,34370,35080,33910,35080,1307310,VCB
2019-01-07,35660,35780,35140,35200,1175810,VCB
2019-01-08,35460,35590,35140,35460,1318810,VCB
...,...,...,...,...,...,...
2024-01-26,91600,92000,91400,92000,953100,VCB
2024-01-29,91800,92000,90900,91200,730500,VCB
2024-01-30,91200,91200,90100,91000,888600,VCB


# Volume

In [5]:
mean_20 = data['volume'].rolling(window=20).mean() #Tính trung bình 20 phiên gần nhất
data['on-balance_volume'] = data['volume']/mean_20 #Tính on-balance volume
data

,open,high,low,close,volume,ticker,on-balance_volume
time,,,,,,,
2019-01-02,34750,35010,34490,34560,1081640,VCB,NaN
2019-01-03,34820,34880,33980,34560,1071350,VCB,NaN
2019-01-04,34370,35080,33910,35080,1307310,VCB,NaN
2019-01-07,35660,35780,35140,35200,1175810,VCB,NaN
2019-01-08,35460,35590,35140,35460,1318810,VCB,NaN
...,...,...,...,...,...,...,...
2024-01-26,91600,92000,91400,92000,953100,VCB,0.643849
2024-01-29,91800,92000,90900,91200,730500,VCB,0.508792
2024-01-30,91200,91200,90100,91000,888600,VCB,0.638869


In [6]:
def label_values(a):
    if pd.isnull(a):
        return a
    elif a < 0.75:
        return 'low'
    elif 0.75 <= a <= 1.25:
        return 'medium'
    else:
        return 'high'
        
data['OBV_label'] = data['on-balance_volume'].apply(label_values)
data.sample(10)

,open,high,low,close,volume,ticker,on-balance_volume,OBV_label
time,,,,,,,,
2022-07-27,62660,63510,62400,63510,1259900,VCB,1.363248,high
2020-05-05,43780,44040,43450,43450,1007640,VCB,0.659627,low
2020-09-15,53990,54320,53670,53670,998970,VCB,0.770756,medium
2023-08-23,88900,88900,86100,86300,1754800,VCB,1.352619,high
2021-04-20,64989,68860,64599,67550,6274200,VCB,3.110089,high
2022-08-16,69430,69430,68670,69010,687100,VCB,0.603940,low
2020-04-21,46190,46510,44100,46510,2159110,VCB,1.405624,high
2019-04-12,44230,44230,43590,43840,355400,VCB,0.357806,low
2020-06-04,57250,57900,57250,57900,450240,VCB,0.342000,low


# Bar

In [7]:
def compare_close_prices(t, t_minus_1):
    if t > t_minus_1:
        return 'up-bar'  
    else:
        return 'down-bar'

data['close_t_minus_1'] = data['close'].shift(1) #Giá đóng cửa ngày t-1

data['bar_type'] = data.apply(lambda row: compare_close_prices(row['close'], row['close_t_minus_1']), axis=1)
data.sample(10)

,open,high,low,close,volume,ticker,on-balance_volume,OBV_label,close_t_minus_1,bar_type
time,,,,,,,,,,
2019-09-05,49780,50810,49780,50030,408900,VCB,0.637069,low,49840.0,up-bar
2022-09-22,65790,66550,65620,66050,1183000,VCB,1.032710,medium,66810.0,down-bar
2020-04-27,45470,45470,43450,43450,2547060,VCB,1.661155,high,44820.0,down-bar
2022-11-22,63080,64349,61390,62240,1936000,VCB,1.458034,high,63510.0,down-bar
2020-03-11,52300,52300,48400,50090,1806190,VCB,1.991497,high,51980.0,down-bar
2023-06-28,84670,86540,84500,85610,821200,VCB,0.885176,medium,84670.0,up-bar
2022-03-01,71550,72310,71210,71970,1295400,VCB,0.776365,medium,71550.0,up-bar
2023-12-14,84000,84500,84000,84400,840600,VCB,0.888391,medium,83700.0,up-bar
2023-01-18,76210,77050,75700,76210,834000,VCB,0.748606,low,75700.0,up-bar


# Spread

In [8]:
data['para'] =abs(data['close'] - data['open'])/(data['high'] - data['low'])
data

,open,high,low,close,volume,ticker,on-balance_volume,OBV_label,close_t_minus_1,bar_type,para
time,,,,,,,,,,,
2019-01-02,34750,35010,34490,34560,1081640,VCB,NaN,NaN,NaN,down-bar,0.365385
2019-01-03,34820,34880,33980,34560,1071350,VCB,NaN,NaN,34560.0,down-bar,0.288889
2019-01-04,34370,35080,33910,35080,1307310,VCB,NaN,NaN,34560.0,up-bar,0.606838
2019-01-07,35660,35780,35140,35200,1175810,VCB,NaN,NaN,35080.0,up-bar,0.718750
2019-01-08,35460,35590,35140,35460,1318810,VCB,NaN,NaN,35200.0,up-bar,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
2024-01-26,91600,92000,91400,92000,953100,VCB,0.643849,low,91000.0,up-bar,0.666667
2024-01-29,91800,92000,90900,91200,730500,VCB,0.508792,low,92000.0,down-bar,0.545455
2024-01-30,91200,91200,90100,91000,888600,VCB,0.638869,low,91200.0,down-bar,0.181818


In [9]:
def label_spread(a):
    if 0 < a < 0.3: return 'low'
    elif 0.3 <= a <= 0.7: return 'medium'
    else: return 'high'

data['label_spread'] = data['para'].apply(label_spread)
data

,open,high,low,close,volume,ticker,on-balance_volume,OBV_label,close_t_minus_1,bar_type,para,label_spread
time,,,,,,,,,,,,
2019-01-02,34750,35010,34490,34560,1081640,VCB,NaN,NaN,NaN,down-bar,0.365385,medium
2019-01-03,34820,34880,33980,34560,1071350,VCB,NaN,NaN,34560.0,down-bar,0.288889,low
2019-01-04,34370,35080,33910,35080,1307310,VCB,NaN,NaN,34560.0,up-bar,0.606838,medium
2019-01-07,35660,35780,35140,35200,1175810,VCB,NaN,NaN,35080.0,up-bar,0.718750,high
2019-01-08,35460,35590,35140,35460,1318810,VCB,NaN,NaN,35200.0,up-bar,0.000000,high
...,...,...,...,...,...,...,...,...,...,...,...,...
2024-01-26,91600,92000,91400,92000,953100,VCB,0.643849,low,91000.0,up-bar,0.666667,medium
2024-01-29,91800,92000,90900,91200,730500,VCB,0.508792,low,92000.0,down-bar,0.545455,medium
2024-01-30,91200,91200,90100,91000,888600,VCB,0.638869,low,91200.0,down-bar,0.181818,low


## Closing bar

In [10]:
def label_close_bar(data):
    q1 = data['low'] + (data['high'] - data['low'])/3
    q2 = data['low'] + 2*((data['high'] - data['low'])/3)
    if data['close'] < q1:
        return 'bottom-third'
    elif q1 <= data['close'] <= q2:
        return 'middle-third'
    else:
        return 'top-third' 

data['close_bar_label'] = data.apply(label_close_bar, axis=1)
data

,open,high,low,close,volume,ticker,on-balance_volume,OBV_label,close_t_minus_1,bar_type,para,label_spread,close_bar_label
time,,,,,,,,,,,,,
2019-01-02,34750,35010,34490,34560,1081640,VCB,NaN,NaN,NaN,down-bar,0.365385,medium,bottom-third
2019-01-03,34820,34880,33980,34560,1071350,VCB,NaN,NaN,34560.0,down-bar,0.288889,low,middle-third
2019-01-04,34370,35080,33910,35080,1307310,VCB,NaN,NaN,34560.0,up-bar,0.606838,medium,top-third
2019-01-07,35660,35780,35140,35200,1175810,VCB,NaN,NaN,35080.0,up-bar,0.718750,high,bottom-third
2019-01-08,35460,35590,35140,35460,1318810,VCB,NaN,NaN,35200.0,up-bar,0.000000,high,top-third
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-01-26,91600,92000,91400,92000,953100,VCB,0.643849,low,91000.0,up-bar,0.666667,medium,top-third
2024-01-29,91800,92000,90900,91200,730500,VCB,0.508792,low,92000.0,down-bar,0.545455,medium,bottom-third
2024-01-30,91200,91200,90100,91000,888600,VCB,0.638869,low,91200.0,down-bar,0.181818,low,top-third


In [13]:
data['close_bar_label'].value_counts()

close_bar_label
top-third       492
bottom-third    428
middle-third    354
Name: count, dtype: int64